In [1]:
from torch.utils.tensorboard import SummaryWriter
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data_path = './data/fmnist'
# данные в виде картинок 28x28
# ToTensor превратит в тензоры 1x28x28
train_dataset = datasets.FashionMNIST(data_path, train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.FashionMNIST(data_path, train=False, download=True, transform=transforms.ToTensor())

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout() # dropout
        self.normlayer = nn.BatchNorm1d(hidden_size) # Слой нормализации (дропаут либо нормализация)
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.normlayer(out)
        out = self.fc2(out)
        return out
    
# выделим из тренировочного датасета 20% данных для валидации
valid_ratio = 0.2
num_valid =  int(valid_ratio * len(train_dataset))
num_train = len(train_dataset) - num_valid

train_dataset, valid_dataset = random_split(dataset=train_dataset, lengths=[num_train, num_valid])
# Делим  датасет на батчи чтобы не предсказывать все 48000 

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

input_size = test_dataset.data[0, :].numel()
hidden_size = 256
num_classes = len(test_dataset.classes)
model = NeuralNet(input_size, hidden_size, num_classes).to(DEVICE)



tb = SummaryWriter()
# print(model.parameters)

tb.add_graph(model, next(iter(train_loader))[0].to(DEVICE))
# tb.close()

In [4]:
def train(model, loader, loss_function, optimizer):
    model.train()

    epoch_loss = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        outputs = model(inputs)

        loss = loss_function(outputs, labels)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(loader)

In [5]:
def test(model, loader, loss_function):
    with torch.no_grad():
        model.eval()
        N = 0
        total_loss = 0.0
        correct = 0.0
        for i, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            N += inputs.shape[0]
            loss = loss_function(outputs, targets)
            total_loss += inputs.shape[0] * loss.item()
            predicted_targets = outputs.argmax(dim=1)
            correct += (predicted_targets == targets).sum().item()
        return total_loss / N, correct / N

In [2]:
tb.

In [8]:
tb = SummaryWriter()

layout = {
    "FashionMNIST": {
        "Loss": ["Multiline", ["loss/train", "loss/valid"]],
        "Accuracy": ["Multiline", ["accuracy/valid", "accuracy/test"]],
    }
}

tb.add_custom_scalars(layout)

In [6]:
input_size = test_dataset.data[0, :].numel()
hidden_size = 256
num_classes = len(test_dataset.classes)
model = NeuralNet(input_size, hidden_size, num_classes).to(DEVICE)

criterion = nn.CrossEntropyLoss() # CCE для задач классификации
optimizer = optim.Adam(model.parameters(), lr=.01)

In [10]:
num_epochs = 5
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    train_loss = train(model, loader=train_loader, loss_function=criterion, optimizer=optimizer)
    val_loss, val_acc = test(model=model, loader=valid_loader, loss_function=criterion)
    

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f'Epoch {epoch}')
    print(f'Train Loss {train_loss:.6f} Val Loss {val_loss:.6f} Val Accuracy {val_acc:.6f}')
    print('---')
    tb.add_scalar("loss/train", train_loss, epoch)
    tb.add_scalar("loss/valid", val_loss, epoch)
    tb.add_scalar("accuracy/valid", val_acc, epoch)
    tb.add_scalar("lr", torch.tensor(optimizer.param_groups[0]["lr"], dtype=torch.float32))

test_loss, test_acc = test(model=model, loader=test_loader, loss_function=criterion)
tb.add_scalar("accuracy/test", test_acc, epoch)
print(f'Test Loss {test_loss:.6f} Test Accuracy {test_acc:.6f}')

Epoch 0
Train Loss 0.278793 Val Loss 0.368799 Val Accuracy 0.868083
---
Epoch 1
Train Loss 0.274911 Val Loss 0.346982 Val Accuracy 0.874667
---
Epoch 2
Train Loss 0.267333 Val Loss 0.342328 Val Accuracy 0.878417
---
Epoch 3
Train Loss 0.259974 Val Loss 0.331535 Val Accuracy 0.884167
---
Epoch 4
Train Loss 0.259119 Val Loss 0.339794 Val Accuracy 0.877917
---
Test Loss 0.359531 Test Accuracy 0.873400


In [ ]:
tb = SummaryWriter()

layout = {
    "FashionMNIST": {
        "Loss": ["Multiline", ["loss/train", "loss/valid"]],
        "Accuracy": ["Multiline", ["accuracy/valid", "accuracy/test"]],
        "Learning_rate": ["Multiline", ["lr"]]
    }
}

tb.add_custom_scalars(layout)